In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm
import ipywidgets as pyw
import hyperspy.api as hys
import tkinter.filedialog as tkf
from tabulate import tabulate
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch import linalg as LA
from torch.utils.tensorboard import SummaryWriter
from torchvision.utils import make_grid
plt.rcParams['font.family'] = 'Times New Roman'

In [ ]:
# create a customized colorbar
color_rep = ["black", "red", "green", "blue", "orange", "purple", "yellow", "lime", 
             "cyan", "magenta", "lightgray", "peru", "springgreen", "deepskyblue", 
             "hotpink", "darkgray"]
print(len(color_rep))
custom_cmap = mcolors.ListedColormap(color_rep)
bounds = np.arange(-1, len(color_rep))
norm = mcolors.BoundaryNorm(boundaries=bounds, ncolors=len(color_rep))
sm = cm.ScalarMappable(cmap=custom_cmap, norm=norm)
sm.set_array([])

cm_rep = ["gray", "Reds", "Greens", "Blues", "Oranges", "Purples"]
print(len(cm_rep))

In [ ]:
def zero_one_rescale(spectrum):
    """
    normalize one spectrum from 0.0 to 1.0
    """
    spectrum = spectrum.clip(min=0.0)
    min_val = np.min(spectrum)
    
    rescaled = spectrum - min_val
    
    if np.max(rescaled) != 0:
        rescaled = rescaled / np.max(rescaled)
    
    return rescaled

In [ ]:
def threed_roll_axis(img):
    stack = np.rollaxis(img, 2, 0)
    return stack

In [ ]:
def data_load(adr, rescale=False, crop=None):
    """
    load a spectrum image
    """
    storage = []
    shape = []
    for i, adr in enumerate(adr):
        temp = hys.load(adr)
        if rescale:
            temp.data = temp.data / np.max(temp.data)
        temp = temp.isig[crop[0]:crop[1]]
        temp = temp.data
        print(np.max(temp), np.min(temp))
        print(temp.shape)

        shape.append(temp.shape)
        storage.append(temp)
    
    shape = np.asarray(shape)
    return storage, shape

In [ ]:
def binning_SI(si, bin_y, bin_x, str_y, str_x, offset, depth, rescale=True):
    """
    re-bin a spectrum image
    """
    si = np.asarray(si)
    rows = range(0, si.shape[0]-bin_y+1, str_y)
    cols = range(0, si.shape[1]-bin_x+1, str_x)
    new_shape = (len(rows), len(cols))
    
    binned = []
    for i in rows:
        for j in cols:
            temp_sum = np.mean(si[i:i+bin_y, j:j+bin_x, offset:(offset+depth)], axis=(0, 1))
            if rescale:
                binned.append(zero_one_rescale(temp_sum))
            else:
                binned.append(temp_sum)
            
    binned = np.asarray(binned).reshape(new_shape[0], new_shape[1], depth)
    
    return binned

In [ ]:
def reshape_coeff(coeffs, new_shape):
    """
    reshape a coefficient matrix to restore the original scanning shapes.
    """
    coeff_reshape = []
    for i in range(len(new_shape)):
        temp = coeffs[:int(new_shape[i, 0]*new_shape[i, 1]), :]
        coeffs = np.delete(coeffs, range(int(new_shape[i, 0]*new_shape[i, 1])), axis=0)
        temp = np.reshape(temp, (new_shape[i, 0], new_shape[i, 1], -1))
        #print(temp.shape)
        coeff_reshape.append(temp)
        
    return coeff_reshape

In [ ]:
file_adr = []

In [ ]:
file_adr.extend(tkf.askopenfilenames())
print(len(file_adr))
print(*file_adr, sep="\n")

In [ ]:
num_img = len(file_adr)
print(num_img)

In [ ]:
# load spectrum images
cr_range = [1.0, 3.0, 0.01] # actual input
data_storage, data_shape = data_load(file_adr, rescale=False, crop=cr_range, roll_axis=False)
print(len(data_storage))
print(data_shape)

e_range = np.arange(cr_range[0], cr_range[1], cr_range[2])
print(len(e_range))

In [ ]:
# actual input
dataset = []
data_shape_new = []
offset = 0
depth = len(e_range)
for img in data_storage:
    print(img.shape)
    processed = binning_SI(img, bin_y, bin_x, str_y, str_x, offset, depth, rescale=True) # include the step for re-scaling the actual input
    print(processed.shape)
    data_shape_new.append(processed.shape)
    dataset.append(processed)
    
data_shape_new = np.asarray(data_shape_new)
print(data_shape_new)

In [ ]:
# create the input dataset
dataset_flat = []
for i in range(num_img):
    dataset_flat.extend(dataset[i].clip(min=0.0).reshape(-1, depth).tolist())
    
dataset_flat = np.asarray(dataset_flat)
print(dataset_flat.shape)

In [ ]:
total_num = len(dataset_flat)
ri = np.random.choice(total_num, total_num, replace=False)

dataset_input = dataset_flat[ri]

In [ ]:
class conv_ae(nn.Module):
    def __init__(self, input_size, encoded_dimension, channels, kernels):
        super(conv_ae, self).__init__()

        self.input_size = input_size
        self.encoded_dimension = encoded_dimension
        
        self.cnn_encoder = nn.Sequential(
            nn.Conv1d(1, channels[0], kernels[0], bias=True),
            nn.BatchNorm1d(channels[0]),
            nn.ReLU(),
            nn.AvgPool1d(2),
            nn.Conv1d(channels[0], channels[1], kernels[1], bias=True),
            nn.BatchNorm1d(channels[1]),
            nn.ReLU(),
            nn.AvgPool1d(2),
            nn.Conv1d(channels[1], channels[2], kernels[2], bias=True),
            nn.BatchNorm1d(channels[2]),
            nn.ReLU(),
            nn.AvgPool1d(2),
            nn.Conv1d(channels[2], channels[3], kernels[3], bias=True),
            nn.BatchNorm1d(channels[3]),
            nn.ReLU(),
            nn.AvgPool1d(2),
            nn.Flatten(),
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(self.encoded_dimension[1], self.input_size, bias=False),
            nn.ReLU(),
        )
    def forward(self, x):
        x = x.view(-1, 1, self.input_size)
        encoded = self.cnn_encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

In [ ]:
if torch.cuda.is_available():
    print("%d gpu available"%(torch.cuda.device_count()))
    cuda_device = torch.device("cuda:1")
    print(torch.cuda.get_device_name(cuda_device))
    torch.cuda.set_device(cuda_device)
else:
    cuda_device = None

In [ ]:
num_comp = 5
channels = [100, 50, 25, num_comp]
kernels = [5, 15, 25, 0]
pooling = [2, 2, 2, 2]

dat_dim = []
tmp_dim = depth
for i in range(len(kernels)):
    tmp_dim += (-kernels[i]+1)
    tmp_dim /= pooling[i]
    dat_dim.append(int(tmp_dim))
    
kernels[-1] = dat_dim[-2] - pooling[-1] + 1
dat_dim[-1] = 1

print(dat_dim)
print(kernels)
print(channels)
print(pooling)

In [ ]:
model = conv_ae(depth, [dat_dim[-1]*channels[-1], num_comp], channels, kernels)
model.cuda(cuda_device)
print(model)

In [ ]:
batch_size = 1600
mini_batches = [dataset_input[k:k+batch_size] for k in range(0, len(dataset_input), batch_size)]

In [ ]:
n_epoch = 100
l_rate = 0.08
lmbd_tv = 0.0005
lmbd_reg = 0
alpha = 1.0

In [ ]:
#optimizer = optim.SGD(model.parameters(), lr=l_rate)
optimizer = optim.Adam(model.parameters())
torch.nn.init.orthogonal_(model.decoder[0].weight)
print(optimizer)

In [ ]:
start = time.time()
ae_coeffs = []
ae_bias = []
for epoch in range(n_epoch):
    for i, m_batch in enumerate(mini_batches):
        
        x = torch.from_numpy(mini_batches[i])
        x = x.to(torch.float32)
        x = x.to(cuda_device)
        x.requires_grad_(requires_grad=False)
        
        encoded, decoded = model(x)
        
        l1_reg = 0
        l2_reg = 0
        tv_reg = 0

        for i in range(num_comp):
            tv_reg += lmbd_tv*LA.norm(torch.gradient(model.decoder[0].weight[:, i])[0], 1)
            l1_reg += lmbd_reg*alpha*(LA.norm(model.decoder[0].weight[:, i], 1))
            l2_reg += lmbd_reg*(1-alpha)*(LA.norm(model.decoder[0].weight[:, i], 2))
        
        #main_loss = F.binary_cross_entropy(decoded, x)
        main_loss = LA.norm((decoded - x), 2) / len(m_batch)
        total_loss = main_loss + l1_reg + l2_reg + tv_reg
        
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()
        
        model.decoder[0].weight.data.clamp_(min=0.0)
        
        if epoch == n_epoch-1:
            coeff_batch = encoded.data.cpu().numpy().tolist()
            ae_coeffs.extend(coeff_batch)            
    
    
    if epoch == 0:
        print(torch.cuda.memory_summary(device=cuda_device))
    
    if (epoch+1) % int(n_epoch/10) == 0:
        print(tabulate([
                        ["epoch", epoch+1], 
                        ["total loss", total_loss.item()],
                        ["-main loss", main_loss.item(), main_loss.item()*100/total_loss.item()],
                        ["-L1 reg.", l1_reg.item(), l1_reg.item()*100/total_loss.item()],
                        ["-L2 reg.", l2_reg.item(), l2_reg.item()*100/total_loss.item()],
                        ["-TV reg.", tv_reg.item(), tv_reg.item()*100/total_loss.item()],
                        ]))
        print("%.2f minutes have passed"%((time.time()-start)/60))
        
        fig, ax = plt.subplots(1, num_comp, figsize=(5*num_comp, 5))
        for i in range(num_comp):
            ax[i].plot(e_range, model.decoder[0].weight.data.cpu()[:, i])
            ax[i].grid()
        fig.tight_layout()
        plt.show()

print("The training has been finished.")

In [ ]:
ae_coeffs = np.asarray(ae_coeffs)
ae_comp_vectors = model.decoder[0].weight.data.cpu().numpy().T
print(ae_coeffs.shape)
print(ae_comp_vectors.shape)

# convert the coefficient matrix into coefficient maps
coeffs = np.zeros_like(ae_coeffs)
coeffs[ri] = ae_coeffs.copy()
coeffs_reshape = reshape_coeff(coeffs, data_shape_new)

In [ ]:
# visualize loading vectors

fig, ax = plt.subplots(1, 1, figsize=(8, 5)) # all loading vectors
for i in range(num_comp):
    ax.plot(e_range, ae_comp_vectors[i], "-", c=color_rep[i+1], label="loading vector %d"%(i+1))
ax.grid()
#ax.legend(fontsize="large")
ax.set_xlabel("eV", fontsize=15)
ax.tick_params(axis="both", labelsize=15)
#ax.axes.get_yaxis().set_visible(False)

fig.tight_layout()
plt.show()

In [ ]:
%matplotlib inline

In [ ]:
%matplotlib qt

In [ ]:
for i in range(num_comp):
    for j in range(num_img):
        print("LV %d map, %d th image"%(i+1, j+1))
        fig, ax = plt.subplots(1, 1, figsize=(6, 5))
        tmp_img = coeffs_reshape[j][:, :, i].copy()
        tmp = ax.imshow(tmp_img, vmin=np.min(coeffs_reshape), vmax=np.max(coeffs_reshape), cmap="viridis")
        ax.axis("off")
        fig.colorbar(tmp, cax=fig.add_axes([0.88, 0.15, 0.04, 0.7]))
        fig.tight_layout()
        plt.show()

In [ ]:
# visualize coefficient maps

if num_img != 1:
    for i in range(num_comp):
        fig, ax = plt.subplots(1, num_img, figsize=(5*num_img, 5))
        for j in range(num_img):
            tmp = ax[j].imshow(coeffs_reshape[j][:, :, i], 
                               vmin=np.percentile(coeffs_reshape[j][:, :, i], 50), cmap="viridis")
            ax[j].set_title("loading vector %d map"%(i+1), fontsize=10)
            ax[j].axis("off")
            
else:
    for i in range(num_comp):
        fig, ax = plt.subplots(1, 1, figsize=(5*num_img, 5))
        tmp = ax.imshow(coeffs_reshape[0][:, :, i], 
                        vmin=np.percentile(coeffs_reshape[0][:, :, i], 50), cmap="viridis")
        ax.set_title("loading vector %d map"%(i+1), fontsize=10)
        ax.axis("off")
        fig.tight_layout()
        fig.colorbar(tmp, cax=fig.add_axes([0.88, 0.15, 0.04, 0.7]))
        plt.show()
        
plt.show()

In [ ]:
import tifffile
for i in range(num_comp):
    tilt_series = []
    for j in range(num_img):
        tilt_series.append(coeffs_reshape[j][:, :, i].astype(np.float32))
    tilt_series = np.asarray(tilt_series)
    tifffile.imsave("tilt_series_latent_%02d.tif"%(i+1), tilt_series)

In [ ]:
# 2D subspace
%matplotlib qt
fig, ax = plt.subplots(1, 1, figsize=(7, 7))

def projection(c1, c2):
    ax.cla()
    ax.scatter(coeffs[:, c1], coeffs[:, c2], s=30, c="black", alpha=0.5)
    ax.grid()
    ax.set_xlabel("loading vector %d"%(c1+1), fontsize=15)
    ax.set_ylabel("loading vector %d"%(c2+1), fontsize=15)
    ax.tick_params(axis="both", labelsize=15)
    fig.canvas.draw()
    fig.tight_layout()

x_widget = pyw.IntSlider(min=0, max=num_comp-1, step=1, value=0)
y_widget = pyw.IntSlider(min=0, max=num_comp-1, step=1, value=1)

pyw.interact(projection, c1=x_widget, c2=y_widget)
plt.show()